# Optical Simulation

In [1]:
from numpy import arcsin,sin,cos,sqrt,exp,pi,array
import enum
Pol = enum.Enum('Pol','s p')

## Snell's Law
\begin{equation*}
 n_{0}\sin\phi_{0} = n_{1}\sin\phi_{1} \\
 \phi_{1} = \sin^{-1} \left(\frac{n_{0}}{n_{1}}\sin\phi_{0}\right)
\end{equation*}

In [2]:
def SnellLaw(n0,n1,angle):
    return arcsin(n0/n1*sin(angle))

## Effective refractive index
\begin{eqnarray*}
 \eta_{j} &=& -n_{j}\cos\phi_{j}  & :s-polarized\\
          &=&\frac{n_{j}}{\cos \phi_{j}} & :p-polarized
\end{eqnarray*}

In [3]:
def EffectiveRefractiveIndex(n,angle,pol):
    if(pol == Pol.s):
        return -n*cos(angle)
    else:
        return n/cos(angle)

## Fresnel coefficient
\begin{equation*}
 r = \frac{\eta_{0}-\eta_{1}}{\eta_{0}+\eta_{1}} \\
 t = \sqrt{\frac{n_{0} \cos \phi_{0}}{n_{1} \cos \phi_{1}}}\cdot\frac{2\sqrt{\eta_{0}\eta_{1}}}{\eta_{0}+\eta_{1}}
\end{equation*}

In [4]:
def FresnelCoefficient(n0,n1,angle0,pol):
    eta0=EffectiveRefractiveIndex(n0,angle0,pol)
    angle1=SnellLaw(n0,n1,angle0)
    eta1=EffectiveRefractiveIndex(n1,angle1,pol)
    r = (eta0-eta1)/(eta0+eta1)
    t = sqrt(n0*cos(angle0)/(n1*cos(angle1)))*2*sqrt(eta0*eta1)/(eta0+eta1)
    return r,t 

## Phase difference
\begin{equation*}
 2\delta = \frac{4\pi}{\lambda}nd\cos\phi
\end{equation*}

In [5]:
def PhaseDifference(n,d,angle,wl):
    return 2*pi*n*d*cos(angle)/wl

## Specific Matrix
\begin{equation*}
M_{j} = \left(
         \begin{array}{cc}
          \cos \delta_{j} & i \eta_{j}^{-1} \sin \delta_{j} \\
          i \eta_{j} \sin \delta_{j}& \cos \delta_{j}\\
         \end{array}
        \right)
\end{equation*}

In [6]:
class Material:
    def __init__(self,n={550:1.5}):
        self.n = n
    def getIndex(self,wl):
        value = self.n.get(wl)
        if value == None:
            nl = self.n.keys()
            l = [x for x in nl if x < wl]
            h = [x for x in nl if x > wl]
            if len(l) != 0 and len(h) != 0:
                x1 = max(l)
                x2 = min(h)
                y1 = self.n.get(x1)
                y2 = self.n.get(x2)
                value = (y2-y1)/(x2-x1)*(wl-x1)+y1
            elif len(l) != 0:
                value = self.n.get(max(l))
            elif len(h) != 0:
                value = self.n.get(min(h))
            else:
                value = None
        return value

In [7]:
class Layer:
    def __init__(self,material,thickness):
        self.material = material
        self.thickness = thickness
    def getSpecificMatrix(self,n0,angle0,wl,pol):
        n = self.material.getIndex(wl)
        angle = SnellLaw(n0,n,angle0)
        delta = PhaseDifference(n,self.thickness,angle,wl)
        eta = EffectiveRefractiveIndex(n,angle,pol)
        return array([[cos(delta),1j/eta*sin(delta)],[1j*eta*sin(delta),cos(delta)]])

## Semi-infinite Model
\begin{equation*}
 R=|r_{1}|^{2} \\
 T=\frac{n_{1} \cos \phi_{1}}{n_{0} \cos \phi_{0}}|t_{1}|^{2} 
\end{equation*}

In [8]:
def SemiInfinite(matIn,matOut,angle0,wl,pol):
    n0 = matIn.getIndex(wl)
    n1 = matOut.getIndex(wl)
    angle1 = SnellLaw(n0,n1,angle0)
    r,t = FresnelCoefficient(n0,n1,angle0,pol)
    rr = abs(r)**2
    tt = n1*cos(angle1)/(n0*cos(angle0))*abs(t)**2
    return {'wl':wl,'ref':rr,'trans':tt}

## Monolayer
\begin{equation*}
 R=\frac{r_{1}+r_{2}\mathrm{e}^{-2i\delta_{1}}}{1+r_{1}r_{2}\mathrm{e}^{-2i\delta_{1}}} \\
 T=\frac{t_{1}t_{2}\mathrm{e}^{-i\delta_{1}}}{1+r_{1}r_{2}\mathrm{e}^{-2i\delta_{1}}}
\end{equation*}

    n0
------------
    n,t
------------
    n1

In [9]:
def Monolayer(matIn,matOut,layer,angle0,wl,pol):
    n0 = matIn.getIndex(wl)
    n1 = matOut.getIndex(wl)
    n = layer.material.getIndex(wl)
    thickness = layer.thickness
    angle = SnellLaw(n0,n,angle0)
    angle1 = SnellLaw(n0,n1,angle0)
    r1,t1 = FresnelCoefficient(n0,n,angle0,pol)
    r2,t2 = FresnelCoefficient(n,n1,angle,pol)
    delta1 = PhaseDifference(n,thickness,angle,wl)
    r = (r1+r2*exp(-2j*delta1))/(1+r1*r2*exp(-2j*delta1))
    t = t1*t2*exp(-2j*delta1)/(1+r1*r2*exp(-2j*delta1))
    rr = abs(r)**2
    tt = n1*cos(angle1)/(n0*cos(angle0))*abs(t)**2
    return {'wl':wl,'ref':rr,'trans':tt}

## Use Specific Matrix
\begin{equation*}
 R=\frac{\eta_{0}(m_{11}+\eta_{N+1}m_{12})-(m_{21}+\eta_{N+1}m_{22})}{\eta_{0}(m_{11}+\eta_{N+1}m_{12})+(m_{21}+\eta_{N+1}m_{22})} \\
 T=\gamma\frac{2\eta_{0}}{\eta_{0}(m_{11}+\eta_{N+1}m_{12})+(m_{21}+\eta_{N+1}m_{22})}
\end{equation*}

\begin{equation*}
\gamma = \left\{
 \begin{array}{l}
  1 & :s-polarized\\
  \frac{\cos\phi_{0}}{\cos\phi_{N+1}} & :p-polarized
 \end{array}
 \right.
\end{equation*}

In [10]:
def SpecificMatrix(matIn,matOut,layer,angle0,wl,pol):
    n0 = matIn.getIndex(wl)
    n1 = matOut.getIndex(wl)
    angle1 = SnellLaw(n0,n1,angle0)
    eta0 = EffectiveRefractiveIndex(n0,angle0,pol)
    eta1 = EffectiveRefractiveIndex(n1,angle1,pol)
    if(pol == Pol.s):
        gamma = 1
    else:
        gamma = cos(angle0)/cos(angle1)
    mat = layer.getSpecificMatrix(n0,angle0,wl,pol)  
    r = (eta0*(mat[0,0]+eta1*mat[0,1])-(mat[1,0]+eta1*mat[1,1]))/(eta0*(mat[0,0]+eta1*mat[0,1])+(mat[1,0]+eta1*mat[1,1]))
    t = gamma*2*eta0/(eta0*(mat[0,0]+eta1*mat[0,1])+(mat[1,0]+eta1*mat[1,1]))
    rr = abs(r)**2
    tt = n1*cos(angle1)/(n0*cos(angle0))*abs(t)**2
    return {'wl':wl,'ref':rr,'trans':tt}

In [11]:
air = Material({550:1.0})
sub = Material({550:2.0})
layer = Layer(Material({550:1.5}),100)

In [12]:
print(SemiInfinite(air,sub,0.2,550,Pol.p))
print(Monolayer(air,sub,Layer(sub,0),0.2,550,Pol.p))
print(SpecificMatrix(air,sub,Layer(sub,0),0.2,550,Pol.p))

{'ref': 0.10664875584891174, 'wl': 550, 'trans': 0.89335124415108835}
{'ref': 0.10664875584891174, 'wl': 550, 'trans': 0.89335124415108835}
{'ref': 0.10664875584891174, 'wl': 550, 'trans': 0.89335124415108802}


In [13]:
print(Monolayer(air,sub,layer,0.2,550,Pol.s))
print(SpecificMatrix(air,sub,layer,0.2,550,Pol.s))

{'ref': 0.0059442899290937483, 'wl': 550, 'trans': 0.99405571007090676}
{'ref': 0.0059442899290937509, 'wl': 550, 'trans': 0.99405571007090643}
